In [ ]:
from huggingface_hub import hf_hub_download, interpreter_login
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import scipy.sparse as sparse

In [ ]:
interpreter_login()

In [ ]:
ts_path_to_file = hf_hub_download(repo_id="qsimeon/celegans_neural_data",
                               repo_type="dataset",
                               filename="worm_data_short.parquet")

connectome_path_to_file = hf_hub_download(repo_id="qsimeon/celegans_connectome_data",
                               repo_type="dataset",
                               filename="consensus_connectome_full_withfunc.csv")

In [ ]:
df = pd.read_parquet(ts_path_to_file)
df = df[(df['worm'] == 'worm9')* (df['source_dataset'] == 'Flavell2023')]
connectome_df = pd.read_csv(connectome_path_to_file)
connectome_df = connectome_df[np.isfinite(connectome_df['functional_weight'])]

In [ ]:
graph = nx.from_pandas_edgelist(connectome_df, 'from_neuron', 'to_neuron', edge_attr='functional_weight')
graph.remove_nodes_from(set(graph.nodes) - set(df['neuron'].unique()))
nodes = np.array(list(graph.nodes()))
nodes_activity =  { str(node): np.array(df[df['neuron'] == node]['original_calcium_data'].tolist()).flatten() for node in nodes}

In [ ]:
lapl = nx.normalized_laplacian_matrix(graph, weight='functional_weight').astype(float)

In [ ]:
vals, vecs = sparse.linalg.eigsh(lapl, k=10, which='SA')
vals, vecs = np.linalg.eigh(lapl.todense())

In [ ]:
from sklearn.decomposition import PCA
coords =  vecs[:, :2]
normalized_coords = PCA(n_components=2, whiten=True).fit_transform(coords)

In [ ]:
for i in range(0, 1000,10 ):
    activities = np.array([nodes_activity[str(node)][i] if len(nodes_activity[str(node)]) > i else np.nan for node in graph.nodes() ])
    plt.scatter(vecs[:, 0], vecs[:, 1], c=activities, cmap='seismic', vmin=-2, vmax=2)
    plt.colorbar()
    plt.show()